# Create data files

In [1]:
import numpy as np
from tqdm.contrib.concurrent import process_map
import h5py
from tqdm import tqdm
from astropy.cosmology import Planck15
from multiprocessing import Pool
from itertools import repeat
from functools import partial

import illustris_python as il

print(np.version.version)

1.24.3


In [4]:
h = 0.6774
kpc_to_km = 3.086e16

# define base path for data
path = "../../sims.TNG/TNG100-1/output/"

lower_limit = 10**11 * h / 1e10
upper_limit = 10**12.5 * h / 1e10

# num = 50000

In [5]:
scale_factor = np.ndarray(100)
redshift = np.ndarray(100)

for snapshot in tqdm(range(100)):
    snap = il.groupcat.loadHeader(path, snapshot)
    
    scale_factor[snapshot] = snap["Time"]  # a
    redshift[snapshot] = snap["Redshift"]  # z
    
time = np.round(Planck15.age(redshift), decimals=3)

with h5py.File("cosmology.hdf5", "w") as file:
    file.create_dataset("a", data=scale_factor)
    file.create_dataset("z", data=redshift)
    file.create_dataset("t", data=time)

100%|██████████| 100/100 [00:04<00:00, 23.68it/s]


In [6]:
# primary subhalo IDs
group_first_sub_ids = il.groupcat.loadHalos(path, 99, fields=["GroupFirstSub"])
group_first_sub_ids = group_first_sub_ids[group_first_sub_ids > -1]

# select centrals in desired mass range
central_halo_mass = il.groupcat.loadSubhalos(path, 99, fields=["SubhaloMassType"])[:,1]
group_first_sub_ids = group_first_sub_ids[(central_halo_mass[group_first_sub_ids] > lower_limit) * (central_halo_mass[group_first_sub_ids] < upper_limit)]
num = len(group_first_sub_ids)

### Load merger trees

In [20]:
def loadTrees(index):
    while True:
        tree = il.sublink.loadTree(path, 99, group_first_sub_ids[index], fields=["SubhaloGrNr", "SnapNum", "SubhaloMassType", "SubhaloBHMdot"], onlyMPB=True)
        if tree is None:
            group_first_sub_ids.pop(index)
        else:
            break
    return tree
    

trees = process_map(loadTrees, range(num), max_workers=100, chunksize=10)
with h5py.File("trees.hdf5", "w") as file:
    for index, tree in enumerate(tqdm(trees)):
        group = file.create_group(str(index))
        group.create_dataset("SubhaloGrNr", data=tree["SubhaloGrNr"])
        group.create_dataset("SnapNum", data=tree["SnapNum"])
        group.create_dataset("SubhaloMassType", data=tree["SubhaloMassType"])
        group.create_dataset("SubhaloBHMdot", data=tree["SubhaloBHMdot"])

  0%|          | 0/14023 [00:00<?, ?it/s]

100%|██████████| 14023/14023 [00:14<00:00, 962.64it/s] 


### Load snapshots

In [21]:
with h5py.File("snapshots.hdf5", "w") as file:
    for snapshot in tqdm(range(100)):
        subhalos = il.groupcat.loadSubhalos(path, snapshot, fields=["SubhaloFlag", "SubhaloGrNr", "SubhaloPos", "SubhaloVel"])
        
        true_halos = subhalos["SubhaloFlag"] == True
        group = file.create_group(str(snapshot))
        group.create_dataset("SubhaloGrNr", data=subhalos["SubhaloGrNr"][true_halos])
        group.create_dataset("SubhaloPos", data=subhalos["SubhaloPos"][true_halos])
        group.create_dataset("SubhaloVel", data=subhalos["SubhaloVel"][true_halos])

100%|██████████| 100/100 [35:18<00:00, 21.19s/it]


### Find satellites for each central

In [22]:
def findSatellites(snapshot, index):
    with h5py.File("trees.hdf5", "r") as trees:
        tree = trees[str(index)]
        if snapshot in np.array(tree["SnapNum"]):
            index_at_snapshot = tree["SubhaloGrNr"][np.array(tree["SnapNum"]) == snapshot][0]
            return list((subhalos == index_at_snapshot).nonzero()[0])
        else:
            return []


# find satellites of every central at every snapshot
with h5py.File("snapshots.hdf5", "r") as snapshots, h5py.File(f"satellites.hdf5", "w") as satellites:
    for snapshot in tqdm(range(100)):
            subhalos = np.array(snapshots[str(snapshot)]["SubhaloGrNr"])
            position = snapshots[str(snapshot)]["SubhaloPos"]
            velocity = snapshots[str(snapshot)]["SubhaloVel"]
            
            with Pool(processes=100) as pool:
                satellite_indices = pool.starmap(findSatellites, zip(repeat(snapshot), range(len(group_first_sub_ids))), chunksize=100)
                  
            snap = satellites.create_group(str(snapshot))
            for index in range(num):
                group = snap.create_group(str(index))
                if len(satellite_indices[index]) != 0:
                    group.create_dataset("SubhaloPos", data=position[satellite_indices[index]])
                    group.create_dataset("SubhaloVel", data=velocity[satellite_indices[index]])
                else:
                    group.create_dataset("SubhaloPos", data=[])
                    group.create_dataset("SubhaloVel", data=[])

100%|██████████| 100/100 [39:14<00:00, 23.54s/it]


In [23]:
def loadGroupKinematics(index):
    with h5py.File("trees.hdf5", "r") as trees:
        history = np.full((2, 100, 3), np.nan)
        for snapshot in range(100):
            if snapshot in np.array(trees[str(index)]["SnapNum"]):
                group_index_at_snapshot = trees[str(index)]["SubhaloGrNr"][np.array(trees[str(index)]["SnapNum"]) == snapshot][0]
                group = il.groupcat.loadSingle(path, snapshot, haloID=group_index_at_snapshot)
                history[0, snapshot] = group["GroupPos"]
                history[1, snapshot] = group["GroupVel"]
    return history


kinematics = np.array(process_map(loadGroupKinematics, range(num), max_workers=100, chunksize=10))
with h5py.File("groups.hdf5", "w") as file:
    for snapshot in tqdm(range(100)):
        snap = file.create_group(str(snapshot))
        snap.create_dataset("GroupPos", data=kinematics[:,0,snapshot])
        snap.create_dataset("GroupVel", data=kinematics[:,1,snapshot])

  0%|          | 0/14023 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:00<00:00, 455.27it/s]


# Calculate quantities from cached data

### Central masses

In [7]:
def loadMass(index):
    with h5py.File("trees.hdf5", "r") as trees:
        tree = trees[str(index)]
        history = np.full((100, 6), np.nan)
        for snapshot in range(100):
            if snapshot in np.array(tree["SnapNum"]):
                history[snapshot] = np.array(tree["SubhaloMassType"])[np.array(tree["SnapNum"]) == snapshot]
    return history


halo_mass = np.array(process_map(loadMass, range(num), max_workers=100, chunksize=10))

  0%|          | 0/14023 [00:00<?, ?it/s]

In [8]:
for halo in tqdm(halo_mass):
    for snapshot in range(100):
        if snapshot > 0 and np.isnan(halo[snapshot, 1]) and not np.isnan(halo[snapshot - 1, 1]):
            halo[snapshot, 1] = (halo[snapshot - 1, 1] + halo[snapshot + 1, 1]) / 2

100%|██████████| 14023/14023 [00:03<00:00, 4554.91it/s]


### Black hole accretion rate

In [9]:
def loadbhmdot(index):
    with h5py.File("trees.hdf5", "r") as trees:
        tree = trees[str(index)] 
        history = np.full(100, np.nan)
        for snapshot in range(100):
            if snapshot in np.array(tree["SnapNum"]):
                history[snapshot] = tree["SubhaloBHMdot"][np.array(tree["SnapNum"]) == snapshot]
    return history


bh_mdot = np.array(process_map(loadbhmdot, range(num), max_workers=100, chunksize=10))

  0%|          | 0/14023 [00:00<?, ?it/s]

### Group occupancy

In [10]:
def calculateSatellites(index):
    with h5py.File("trees.hdf5", "r") as trees, h5py.File(f"satellites.hdf5", "r") as subhalos:
        history = np.full(100, np.nan)
        for snapshot in range(100):
            if snapshot in np.array(trees[str(index)]["SnapNum"]):
                history[snapshot] = subhalos[str(snapshot)][str(index)]["SubhaloPos"].shape[0] - 1
    return history


satellites = np.array(process_map(calculateSatellites, range(num), max_workers=100, chunksize=10))

  0%|          | 0/14023 [00:00<?, ?it/s]

### Finding the radial and tangential velocities
Suppose we have some velocity vector in cartesian coordinates:
$$\vec{v}=v_x\hat{x}+v_y\hat{y}+v_z\hat{z}.$$
We want to convert this to a representation in spherical coordinates, given by
$$\vec{v}=v_r\hat{r}+v_\theta\hat{\theta}+v_\phi\hat{\phi}.$$

Then the radial velocity component is simply $v_r$, and the tangential velocity component is given by
$$v_\text{tan}=\sqrt{v_\theta^2+v_\phi^2}.$$

Notice that we can extract the spherical components by projecting onto the spherical unit vectors.
For this we need the dot products between cartesian and spherical unit vectors:
$$\hat{x}\cdot\hat{r}=\sin\theta\cos\phi, \quad \hat{y}\cdot\hat{r}=\sin\theta\sin\phi, \quad \hat{z}\cdot\hat{r}=\cos\theta,$$
$$\hat{x}\cdot\hat{\theta}=\cos\theta\cos\phi, \quad \hat{y}\cdot\hat{\theta}=\cos\theta\sin\phi, \quad \hat{z}\cdot\hat{\theta}=-\sin\theta,$$
$$\hat{x}\cdot\hat{\phi}=-\sin\phi, \quad \hat{y}\cdot\hat{\phi}=\cos\phi, \quad \hat{z}\cdot\hat{\phi}=0.$$

Hence the spherical components are
$$v_r=\vec{v}\cdot\hat{r}=v_x\sin\theta\cos\phi+v_y\sin\theta\sin\phi+v_z\cos\theta,$$
$$v_\theta=\vec{v}\cdot\hat{\theta}=v_x\cos\theta\cos\phi+v_y\cos\theta\sin\phi-v_z\sin\theta,$$
$$v_\phi=\vec{v}\cdot\hat{\phi}=-v_x\sin\phi+v_y\cos\phi.$$

Now, bearing in mind the ordinary coordinate transformations:
$$x=r\sin\theta\cos\phi, \quad y=r\sin\theta\sin\phi, \quad z=r\cos\theta$$
$$\implies r=\sqrt{x^2+y^2+z^2}, \quad \rho=\sqrt{x^2+y^2}=r\sin\theta,$$
we get
$$v_r=\vec{v}\cdot\hat{r}=v_x\frac{x}{r}+v_y\frac{y}{r}+v_z\frac{z}{r},$$
$$v_\theta=\vec{v}\cdot\hat{\theta}=v_x\frac{xz}{\rho r}+v_y\frac{yz}{\rho r}-v_z\frac{\rho}{r},$$
$$v_\phi=\vec{v}\cdot\hat{\phi}=-v_x\frac{y}{\rho}+v_y\frac{x}{\rho}.$$

In [11]:
def loadVelocities(index):
    with h5py.File("trees.hdf5", "r") as trees, h5py.File(f"satellites.hdf5", "r") as subhalos, h5py.File("groups.hdf5", "r") as groups:
        history = np.full(100, np.nan)
        for snapshot in range(100):
            if snapshot in np.array(trees[str(index)]["SnapNum"]) and len(subhalos[str(snapshot)][str(index)]["SubhaloPos"]) > 1:
                    
                x = subhalos[str(snapshot)][str(index)]["SubhaloPos"][1:] * scale_factor[snapshot] * kpc_to_km / h  # [km]
                v = subhalos[str(snapshot)][str(index)]["SubhaloVel"][1:]  # [km/s]
                
                x_com = groups[str(snapshot)]["GroupPos"][index] * scale_factor[snapshot] * kpc_to_km / h  # [km]
                v_com = groups[str(snapshot)]["GroupVel"][index] / scale_factor[snapshot]  # [km/s]

                x -= x_com
                v -= v_com
                
                # at some point between numpy v1.19.5 and v1.24.3 the handling of dtypes has changed so this is now required
                x = x.astype("float64")
                v = v.astype("float64")
                
                rho = np.sqrt(x[:,0]**2 + x[:,1]**2)
                r = np.sqrt(rho**2 + x[:,2]**2)

                v_r = (v[:,0]*x[:,0] + v[:,1]*x[:,1] + v[:,2]*x[:,2]) / r
                theta_dot = ((v[:,0]*x[:,0]*x[:,2] + v[:,1]*x[:,1]*x[:,2]) / rho - v[:,2]*rho) / r
                phi_dot = (-v[:,0]*x[:,1] + v[:,1]*x[:,0]) / rho
                v_tan_2 = theta_dot**2 + phi_dot**2

                beta = 1 - v_tan_2 / (2*v_r**2)

                history[snapshot] = np.median(beta)
    return history


beta = np.array(process_map(loadVelocities, range(num), max_workers=100, chunksize=10))

  0%|          | 0/14023 [00:00<?, ?it/s]

In [16]:
# scale to correct units and save
bh_mdot_scaled = bh_mdot * 1e10 / 0.978  # [M_sol/Gyr]

halo_mass[halo_mass == 0] = 1e-15
halo_mass_scaled = np.log10(halo_mass * 1e10 / h)  # [log10(M_sol)]

with h5py.File("halo_properties.hdf5", "w") as file:
    file.create_dataset("bh_mdot", data=bh_mdot_scaled)
    file.create_dataset("satellites", data=satellites)
    file.create_dataset("beta", data=beta)
    file.create_dataset("Mh", data=halo_mass_scaled[:,:,1])
    file.create_dataset("Ms", data=halo_mass_scaled[:,:,4])
    file.create_dataset("Mg", data=halo_mass_scaled[:,:,0])